In [5]:
from astropy.table import Table
import numpy as np
import matplotlib.pyplot as plt
import mtl
import quickcat
from importlib import reload

%matplotlib inline

In [3]:
# global truth file
truth = Table.read("targets/subset_truth_dr8_mtl_dark_gray_NGC.fits")

# This is the input to fiberassign
old_mtl = Table.read("strategy_A/targets/full_subset_dr8_mtl_dark_gray_northern_cap.fits")

# This is the results of fiberassign
fibertiles = glob.glob("strategy_A/fiberassign_gray/tile-*.fits")

# This is the final zcat after the run
zcat = Table.read("strategy_A/zcat/gray_zcat.fits")

# This is the new mtl after using zcat
new_mtl = Table.read("strategy_A/targets/dark0_subset_dr8_mtl_dark_gray_northern_cap.fits")

In [6]:
print(len(truth), len(old_mtl), len(new_mtl), len(zcat), len(fibertiles))

4139902 4139902 4139902 218682 48


In [6]:
mtl = reload(mtl)
quickcat = reload(quickcat)